# 목적

- 파이토치로 CNN을 구현하여 이미지 데이터가 어떤 이미지인지 분류하는 모델을 생성하겠다
- 객체지향 방식으로 모델을 생성
- 데이터
  - 치매/정상 뇌스캔 이미지
  - tarin/ad or noraml : 80개
  - test/ad or noaml : 60개
  - 형식 *.jpg
- 목표
  - 뇌스캔 사진 입력 -> 치매를 진단(정상/치매)
  - 단, 데이터가 적어서 모델의 신뢰성은 얻기 어렵다

# 모듈가져오기, 환경변수

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 참고 (연습용 내부데이터) -> FashionMNIST
from torchvision import transforms, datasets

In [ ]:
# GPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

## 학습 관련 용어 정리

- 종류 1
  - 온라인 학습 : 학습된 모델이 실시간 반영 ( 리스크가 크다 )
  - 오프라인 학습 : 모델 교체시 시스템을 다운 -> 교체 -> 가동, 실제 서비스와 분리되어 있다
- 종류 2
  - 배치학습
    - 전체 데이터를 통으로 학습에 한 번에 넣어서 진행 (많은 리소스를 요구한다)
  - 미니배치학습
    - 전체 데이터를 n개로 분할해서 학습 횟수를 늘려서 진행
- 전략
  - 내가 직접 신경망을 만들어서 학습
  - 전이학습(tranfer learning)
    - 타인이 오랜시간 /대략의 데이터로 학습한(사전학습) 모델을 가져와서 사용하는 전략
    - 파인튜닝
    - 프럼프트 러닝
    - 인컨텍스트 러닝(최신 유행 전략)
      - 제로샷 러닝
      - 원샷 러닝
      - 퓨샷 러닝
- 학습 시 요소
  - 에포크 : 전체 데이터를 한 번 다 사용하여 학습한 경우 1에폭이라고 표현 (1세대 학습)
  - 배치사이즈 : 1회 학습 시 투입되는 데이터량 -> 미니 배치 학습
  - 예시, 데이터가 총 1,000,000개가 있다
  - 1회 학습시 1,000개를 사용 (세대별로 1,000개의 조합이 다르다)
  - 10에폭으로 학습하겠다
  - 1에폭 학습시 학습 행위가 몇 번 진행되는가? -> 1,000번 학습이 진행
  - 조기학습 종료 (early_stopping)
    - 더 이상 학습의 성과가 없으면, 학습 행위를 중단한다

In [ ]:
# 환경변수(학습에 관련된 설정 값)
# GPU환경이면 30회, CPU면 5회만 진행
EPOCHES = 30 if torch.cuda.is_available() else 5
BATCH_SIZE = 64 # 1회 학습시 64개의 데이터를 공급받아서 학습하겠다

# 데이터 준비

- 데이터 공급자를 설계 제공

In [ ]:
# 훈련용 데이터 공급자
train_loader = torch.utils.data.DataLoader(
  datasets.FashionMNIST(
    root      = './data',   # 데이터가 저장된 위치
    train     = True,       # 훈련용
    download  = True,       # 부재시 다운로드 진행
    # 이미지를 전처리(가공, 등등)
    transform = transforms.Compose([
      transforms.ToTensor(), # 이미지는 텐서로 공급받겟다
      transforms.Normalize( 0.2, 0.3 ) # 정규화, 평균, 표준편차 설정값
    ])      
  ),
  batch_size = BATCH_SIZE, 
  shuffle = True
)

In [ ]:
test_loader  = torch.utils.data.DataLoader(
  datasets.FashionMNIST(
    root      = './data',   # 데이터가 저장된 위치
    train     = False,      # 테스트용
    download  = True,       # 부재시 다운로드 진행
    # 이미지를 전처리(가공, 등등)
    transform = transforms.Compose([
      transforms.ToTensor(), # 이미지는 텐서로 공급받겟다
      transforms.Normalize( 0.2, 0.3 ) # 정규화, 평균, 표준편차 설정값
    ])   
  ),
  batch_size = BATCH_SIZE,
  shuffle = True
)

# 합성곱 요소간 관계식

- 합성곱을 통과한 feature map의 크기
- H = ((h + 2*p-k)/s) + 1
- W = ((w + 2*p-k)/s) + 1

# 신경망 구성

# 인공신경망 구성

In [ ]:
# class Net(nn.Module):
#   # 멤버 변수
#   # 멤버 함수 (상속받은 함수를 재정의 or 내가 직접 구현)
#   def forward(slef, x):
#     # 부모로부터 상속받은 함수, 모델의 순방향으로 네트워크를 구성하는 함수
#     # 모델을 순방향으로 네트워클를 구성하는 함수
#     # 네트워크 연결(add())
#     pass
#   # 생성자 (self <-> this)
#   def __init__(self):
#     # 1. 부모 생성자 호출 -> 통상 상속받으면 진행
#     super(Net, self).__init__()
#     # 네트워크의 각 요소를 생성 -> 합성곱층, 풀링층, 전결합층, 출력층
#     # MNIST(28, 28) -> (14, 14) -> (7, 7) -> FC -> 출력층 수렴
#     # 1F 합성곱층 1개 -> 멤버변수 -> 객체멤버
#     # 1개 이미지 -> 합성곱 통과시키면 32장으로 증폭
#     self.conv1 = nn.Conv2d(1, 32, 5, 1, padding='same')
#     # 2F 합성곱층 1개
#     self.conv2 = nn.Conv2d(32, 32*32, 5, 1, padding='same')
#     # 과적합 방지 층 -> 학습 방해, 10% 방해하겠다 -> 학습이 안되게 조절
#     self.dropout = nn.Dropout2d(0.1)
#     # 3F FC -> 4D => 2D로 Flattern 한 후에 출력에 수렴하기 전 적당한 크기로 완충하는 역할
#     # 원본 이미지 784pixel => 3136pixel => 1024 <- feature
#     self.fc = nn.Linear(7*7*32*2, 1024)
#     # 4F output 생성
#     self.output = nn.Linear(1024 ,10)
#     pass

In [ ]:
class Net(nn.Module):
  def forward(self, x): # 순전파신경망(x->y)
    x = F.relu( F.max_pool2d( self.conv1( x ), 2) )                 # 28 -> 14
    x = F.relu( F.max_pool2d( self.dropout( self.conv2( x ) ), 2) ) # 14 -> 7
    x = x.view( -1 , 7*7*32*2  ) # 이미지가 2번 50%씩 줄어서 사이즈가 7
    x = F.relu( self.fc( x ) ) # 3136 -> 1024 (feature size)
    x = F.dropout( x, training=self.training ) # 학습률로표시하는 예
    x = F.relu( self.output(x) ) # 1024 -> 10   
    return F.log_softmax( x, dim=1 )

  def __init__(self): # 32, 32*2, 1024, 5, 1 모두 설정값
    super(Net, self).__init__()
    self.conv1   = nn.Conv2d(1, 32,    5, 1, padding='same' )        
    self.conv2   = nn.Conv2d(32, 32*2, 5, 1, padding='same' )
    self.dropout = nn.Dropout2d(0.1)
    self.fc      = nn.Linear(7*7*32*2, 1024 )        
    self.output  = nn.Linear(1024, 10 )

In [ ]:
# 모델 생성 -> GPU 적용
model = Net().to(DEVICE)

In [ ]:
# 파라미터 확인 -> w, b
model.parameters(), model

(<generator object Module.parameters at 0x7f6d7204ae50>, Net(
   (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
   (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
   (dropout): Dropout2d(p=0.1, inplace=False)
   (fc): Linear(in_features=3136, out_features=1024, bias=True)
   (output): Linear(in_features=1024, out_features=10, bias=True)
 ))

In [ ]:
# 최적화 도구 준비 -> 케라스(compile() 연결된다)
# ( 튜닝대상(W,b), 학습계수(적용비율), 속도 )
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5 )

# 학습 구성

# 학습 실행 및 테스트

In [ ]:
def train(model, trian_loader, optimizer, epoch):
  '''
    model : 모델 (학습대상)
    train_loader : 학습시 사용 할 데이터를 제공하는 공급자
    optimizer : 모델 학습시 손실 값을 최소로 줄이겠끔 조정하는 주체 -> 최적의 W,b를 계싼(조정, 수정)하는 주체
  '''
  # 학습 모드로 전환
  model.train()
  # 미니 배치 학습 진행 -> 1회 반복시 64개 데이터를 추출(batch_size만큼)
  for idx, (data, target) in enumerate(train_loader):
    # data : 훈련용 이미지 (tensor [64,]), target: 정답 
    '''
        - 데이터 형태
        torch.Size([64, 1, 28, 28]) tensor([9, 3, 7, 2, 6, 6, 5, 4, 9, 7, 9, 5, 9, 1, 3, 1, 1, 8, 5, 2, 6, 9, 7, 0,
        4, 9, 3, 9, 7, 9, 8, 9, 3, 5, 8, 6, 2, 2, 4, 8, 9, 6, 1, 4, 6, 9, 3, 2,
        2, 7, 7, 6, 1, 4, 7, 7, 2, 6, 7, 6, 9, 3, 2, 2])

    '''
    # print(data.size(), target)
    # 1. 학습에 사용되는 데이터를 gpu 혹은 cpu 사용으로 설정
    data = data.to(DEVICE)
    target = target.to(DEVICE)
    # 2. 최적화 도구에 남아 있는 값을 초기화한다 (이전 세대에서 사용한 잔재를 초기화한다)
    optimizer.zero_grad()
    # 3. 모델에 데이터를 삽입 -> 출력 획득 -> 예측 수행되었다
    output = model(data)
    # print(output.size()) # (64, 10) : torch.Size([64, 10])
    # 4. 정답과 예측값을 비교해서 최적화 수행
    # cross_entropy() : 손실함수
    loss = F.cross_entropy(output, target)
    # 5. 최적화 수행 -> 오차역전파 수행(y->x) -> 이를 위해 w, b를 조정
    loss.backward()
    # 6. 최적화된 도구 모델 파라미터에 최종 반영
    optimizer.step()
    # 로그 출력 세대별 로그
    if idx % 200 == 0:
      # 학습세대 회차/세개별 학습 총회차 손실값
      print(f'Epoch : {epoch}, {idx*len(data)} / {len(train_loader)} loss={loss.item()}')
    # break
  pass

In [ ]:
def test(model, test_loader):
  # 테스트 모드로 전환
  model.eval()
  loss = 0
  acc = 0
  # 테스트 행위간 발생되는 모든 내용을 기록하지 않는다
  with torch.no_grad(): # 기록하지 말아라, 모델에 영향을 미치지 않게 처리해라
    for data,target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)
      # 손실값 획득
      loss = F.cross_entropy(output, target, reduction='sum').item()
      # 예측값 추출 -> 최고값을 가진 인덱스 추출 => numpy.argmax() : 특정배열에서 최고값을 가진 방의 번호
      pred = output.max(1, keepdim=True)[1]
      # 정확도 체크 -> 행렬 비교 -> True => 1 => sum
      acc += pred.eq(target.view_as(pred)).sum().item()
      pass
    pass
    # 평균 손실 (값)
    mean_loss = loss / len(test_loader.dataset)
    # 평균 정확도 (%)
    mean_acc = acc / len(test_loader.dataset) * 100
    return mean_loss, mean_acc
  pass

In [ ]:
# 조기학습 종료 컨셉이 없으므로 EPOCHES 설정횟수만큼 반복
for epoch in range( 1, EPOCHES+1 ):
  # 1세대 학습이 끝나면 바로 테스트 
  # 훈련
  train( model, train_loader,   optimizer, epoch )
  # 테스트
  loss, acc = test( model, test_loader )
  # 로그
  print( f'epoch:{epoch}, 손실:{loss}, 정확도:{acc}' )
  # 조기학습 종료 추가(전체 세대 학습을 채울 필요는 없다), 여기서는 생략
  # break

Epoch : 1, 0 / 938 loss=2.3045670986175537
Epoch : 1, 12800 / 938 loss=0.892511248588562
Epoch : 1, 25600 / 938 loss=0.7045785188674927
Epoch : 1, 38400 / 938 loss=0.7672329545021057
Epoch : 1, 51200 / 938 loss=0.38210806250572205
epoch:1, 손실:0.0011154960632324218, 정확도:83.31
Epoch : 2, 0 / 938 loss=0.6519713997840881
Epoch : 2, 12800 / 938 loss=0.29530903697013855
Epoch : 2, 25600 / 938 loss=0.3511064350605011
Epoch : 2, 38400 / 938 loss=0.20518946647644043
Epoch : 2, 51200 / 938 loss=0.34591400623321533
epoch:2, 손실:0.00026280245780944825, 정확도:85.52
Epoch : 3, 0 / 938 loss=0.39360710978507996
Epoch : 3, 12800 / 938 loss=0.4840443432331085
Epoch : 3, 25600 / 938 loss=0.2773706614971161
Epoch : 3, 38400 / 938 loss=0.3669925928115845
Epoch : 3, 51200 / 938 loss=0.38914960622787476
epoch:3, 손실:0.001005734920501709, 정확도:87.25
Epoch : 4, 0 / 938 loss=0.3977683186531067
Epoch : 4, 12800 / 938 loss=0.4534311294555664
Epoch : 4, 25600 / 938 loss=0.3920537829399109
Epoch : 4, 38400 / 938 loss=0.

In [ ]:
# 학습 결과 시각화 > 